Training and Testing with ( Acceleration + ECG) Mhealth & Movesense datasets

In [ ]:
# Importing libraries
import pandas as pd
import seaborn as sns
import numpy as np
import os
import glob
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
%matplotlib inline
from keras import layers
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Load training data

In [ ]:
# File directory_path
## Columns of the training dataset: 'subject column' was introduced to extract activities performed by a specific subject
headers=('acc_CHsensor_X','acc_CHsensor_Y','acc_CHsensor_Z','ecg_i','ecg_ii','acc_LAsensor_X','acc_LAsensor_Y','acc_LAsensor_Z',
       'gyro_LAsensor_X','gyro_LAsensor_Y','gyro_LAsensor_Z','magn_LAsensor_X','magn_LAsensor_Y','magn_LAsensor_Z',
      ' acc_RLAsensor_X',' acc_RLAsensor_Y',' acc_RLAsensor_Z', 'gyro_RLAsensor_sensor_X',' gyro_RLAsensor_sensor_Y',
      ' gyro_RLAsensor_sensor_Z','magn_RLAsensor_X','magn_RLAsensor_Y','magn_RLAsensor_Z','classes', 'subject')
# File location
path = r"C:/Users/val-c/Machine Learning_Deep Learning/MHEALTHDATASET-csv/*.csv" # get files from the introduced subjects, .csv
data_files = glob.glob(path)
# Get list of files from path
data_frames = []
for file in data_files:
    movement_activity_data = pd.read_csv(file,header=None, names=headers)
    data_frames.append( movement_activity_data)

In [ ]:
# Merge the folder files and display data
movement_activity_data = pd.concat(data_frames, ignore_index=True)
movement_activity_data.head() #display first-five rows

In [ ]:
# Examining the class label imbalance
counts = np.bincount(movement_activity_data["classes"])
neg , pos = counts[0], counts[1]
total = neg + pos
print('samples:\nTotal: {}\nPositive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))

In [ ]:
# Plot the number of samples present in each class_label using a Bar plot.
class_label = ['Null','Standing still','Sitting and relaxing','Lying down','Walking', 'Climbing stairs',
                       'Waist bends forward', 'Frontal elevation of arms','Knees bending (crouching)','Cycling',
                      'Jogging','Running','Jump front & back']
plt.figure(figsize=(8, 7), dpi=80) # figure sizing
num_classes = len(class_label)
plt.bar(range(num_classes), movement_activity_data["classes"].value_counts(), color='k', linewidth=0.1) #plot data
# Replace the numerical x-axis ticks with class labels
plt.xticks(range(num_classes), class_label, rotation=45, ha='right')
# the x-axis and y-axis labels, title
plt.xlabel('\nClasses', fontsize=12)
plt.ylabel('\nCount', fontsize=12)
plt.title("\nNumber of samples by class", fontsize=14)
plt.show() #display plot

In [ ]:
# Downsampling null_classes_0 observation to account for the imbalance distribution
def downsample_data(movement_activity_data):
    mask = movement_activity_data['classes'] == 0
    HAR_data_classes0 = movement_activity_data[mask].sample(n=30720, random_state=1)
    HAR_data_classes_else = movement_activity_data[~mask]
    return pd.concat([HAR_data_classes0, HAR_data_classes_else], ignore_index=True)
movement_data = downsample_data(movement_activity_data)
# display the statistical balance
movement_data.classes.value_counts()

In [ ]:
# Feature aggregation of the ECG measurements by creating a new column of the average of ecg_i and ii called 'Mean_ecg'
movement_data.insert(5, 'Mean_ecg', (movement_data['ecg_i'] + movement_data['ecg_ii']) / 2)
# Display the DataFrame
movement_data.head(n=2) # display first-two rows

In [ ]:
# Drop other columns not needed for the input
move_data=movement_data.drop(['ecg_i','ecg_ii','acc_LAsensor_X','acc_LAsensor_Y',
                                       'acc_LAsensor_Z', 'gyro_LAsensor_X','gyro_LAsensor_Y','gyro_LAsensor_Z',
                                       'magn_LAsensor_X','magn_LAsensor_Y','magn_LAsensor_Z',' acc_RLAsensor_X',
                                       ' acc_RLAsensor_Y',' acc_RLAsensor_Z', 'gyro_RLAsensor_sensor_X',
                                       ' gyro_RLAsensor_sensor_Y',' gyro_RLAsensor_sensor_Z','magn_RLAsensor_X',
                                       'magn_RLAsensor_Y','magn_RLAsensor_Z','subject'],axis=1)
# display dataframe
move_data.head(n=2) # display first-two rows

In [ ]:
# Check for missing values
move_data.isna().sum(axis=0).to_frame('Total')

In [ ]:
# Display the statistics of the dataframe
move_data.describe().T

- Filtering the training data

In [ ]:
# Columns of the dataframe to apply filtering
cols=['acc_CHsensor_X','acc_CHsensor_Y','acc_CHsensor_Z','Mean_ecg']
# filtering the sensor signal from noise
# low pass filtering > remove high frequency noise
## Defining the low-pass filter parameters
cutoff_freq = 20 # hz
fs = 50  # hz
nyquist_freq = 0.5 * fs
n_order = 3 #filter_order
# Designing the Butterworth filter
b, a = butter(n_order , cutoff_freq/nyquist_freq, btype='low')
# Applying the filter to the selected data_columns
filtered_data = filtfilt(b, a,move_data[cols].values, axis=0)
#assign the filtered data to the dataframe
filtered_movement_data= pd.DataFrame(filtered_data, columns=cols) # put the filtered data to df
filtered_movement_data[['classes']] = move_data[['classes']] # assign back the class labels and subject
# Display the filtered DataFrame
filtered_movement_data.head() # display first_five rows

- Feature selection

In [ ]:
# Selected features and class from the dataframe
Features =filtered_movement_data.copy() #features/input
Label = Features.pop('classes') # labels/output

In [ ]:
# Display columns of the selected features (input)
Features.columns

- Data segmentation

In [ ]:
# Creating a function for the sliding_window  ( this function would be applied to the training set_Mhealth and testing set_movesense data)
#num_time_step: which specifies the length of each segment
#step_size: which specifies the step size used to slide the window over the data
def create_sequences(X, y, num_time_step, stepsize=1): # X:input y: output/target
    segment, label = [],[]
    for i in range(0,len(X) - num_time_step+1, stepsize):
        x = X[i:i + num_time_step]
        labels = y[i + num_time_step-1]
        segment.append(x)
        label.append(labels)
    return np.array(segment), np.array(label)

- Normalize the training data

In [ ]:
# StandardScaler to normalize the input variables
scaler = StandardScaler()
# Normalize the training input
X_train_normalized = scaler.fit_transform(Features)

In [ ]:
# Creating the training sequence by applying the above create_sequences function
X_train_seq,y_train_seq = create_sequences(X_train_normalized, Label, num_time_step=100, stepsize=50)
print (X_train_seq.shape, y_train_seq.shape)

In [ ]:
# Extract window_length,number of features, number of classes to be used in the input & ouput of the neural network
window_length,num_features,num_classes= X_train_seq.shape[1], X_train_seq.shape[2],len(np.unique(y_train_seq))
print(window_length,num_features,num_classes)

- Configure the neural network

In [ ]:
# Configuring >> CNN-1D_LSTM network architecture
input_shape=(window_length,num_features)
model= keras.Sequential([
# Input layer
     layers.Input(input_shape), #input
     layers.Conv1D(filters=64, kernel_size=3, padding='same'),# CNN-1D layer 1
     layers.BatchNormalization(),# batch_normalization
     layers.Activation('relu'), # activation function

# Hidden Conv Layer
     layers.Conv1D(filters=64, kernel_size=3, padding='same'), # CNN-1D layer 2
     layers.BatchNormalization(), # batch_normalization
     layers.Activation('relu'),# activation function

# Recurrent LSTM Layers
     layers.LSTM(units=128, return_sequences=True), # LSTM layer 1
     layers.Activation('relu'), # activation function
     layers.LSTM(units=128,return_sequences=False), # LSTM layer 2
     layers.Activation('relu'), # activation function

# Dense layers
     layers.Dense(128,activation="relu"), # Dense layer 1
# Activation function for the output 'softmax' for the Multi-classification
     layers.Dense(num_classes, activation='softmax')]) # ouput of dense layer

# Summary of the model configuration
model.summary(line_length=None,
              positions=None,
              print_fn=None,
              expand_nested=False,
              show_trainable=False,
              layer_range=None,
             )

- Compile the model

In [ ]:
# Compiling the model
initial_learning_rate =0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10,
    decay_rate=0.9) #decay_rate/factor
opt = keras.optimizers.RMSprop(learning_rate=lr_schedule) # optimizer
model.compile(optimizer=opt, loss= keras.losses.SparseCategoricalCrossentropy(), metrics=["sparse_categorical_accuracy"],)

- Train the model

In [ ]:
# Training of the subsets for 10 iterations/epochs
history=model.fit(X_train_seq, y_train_seq,validation_split=0.1,epochs=10,batch_size=64)

In [ ]:
# View the training history in pandas df
train_history= pd.DataFrame(history.history)
train_history #display history

In [ ]:
# Training progress / session visualization
# Training accuracy plotting
plt.figure(figsize=(8, 7), dpi=75)
plt.subplot(1, 2, 1)
plt.plot(history.history['sparse_categorical_accuracy'],c='g') #plot accuracy
plt.plot(history.history['val_sparse_categorical_accuracy'],c='g', linestyle='--' ) #plot val_acc
plt.title('Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['accuracy', 'val_accuracy'])
# Training loss plotting
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],c='r') #plot loss
plt.plot(history.history['val_loss'],c='r',linestyle='--') #plot val_loss
plt.title('Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['loss','val_loss'])
#display plot
plt.tight_layout()
plt.show()

- Evaluate the training session

In [ ]:
# Training session evaluation of the model
loss, accuracy =model.evaluate(X_train_seq,y_train_seq,verbose=0)
print(f'Train loss: {loss:.4f}')
print(f'\nTrain accuracy: {accuracy:.4f}')

In [ ]:
# Prediction on the train_data to get training classifications
classify_train_data =model.predict(X_train_seq,verbose=0)
classify_train_data= np.argmax(classify_train_data, axis=1)

In [ ]:
# Create confusion matrix for the classifications on the training session
cm = confusion_matrix(y_train_seq, classify_train_data)
class_labels = ['null','Standing still','Sitting and relaxing','Lying down','Walking', 'Climbing stairs',
                       'Waist bends forward', 'Frontal elevation of arms','Knees bending (crouching)', 'Cycling',
                      'Jogging','Running','Jump front & back']
# Plot the confusion matrix for the classification >training
plt.figure(figsize=(12, 8),dpi=80)
sns.heatmap(cm,fmt="d",annot=True, cmap='Blues', xticklabels=class_labels, yticklabels=class_labels,linewidths = 0.1)
plt.title('\nConfusion Matrix : Classification-training', fontsize=14, fontweight='bold')  #title
plt.xlabel('\nPredicted Activities', fontsize=12, fontweight='bold') #x-axis label
plt.ylabel('\nActual Activities', fontsize=12, fontweight='bold') #y-axis label
#display cm plot
plt.show()

# Load test data

- Load the ECG dataset of the Movesense

In [ ]:
# ecg files > load the ECG files from folder
# files folder_location
folder_path_ecg ='C:/Users/val-c/Desktop/movesense_lblled_rsp_ecg'
# Get the list of all files in the folder_location
file_list = os.listdir(folder_path_ecg)
# Loop through the list of files and read them into pandas
dataframes= []
for file_name in file_list:
    if file_name.endswith(".csv"):  # files in csv format
        file_path = os.path.join(folder_path_ecg, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
# Combine all the data_frame into one
df_ecg = pd.concat(dataframes, ignore_index=True)
df_ecg.head(n=5)# display dataframe for first-five rows

In [ ]:
#display nos of row & columns
#df_ecg.shape

In [ ]:
# Save the merged DataFrame(df_ecg) to a CSV file
#df_ecg.to_csv('ecg_file.csv', index=False)

- Load the IMU dataset of the Movesense

In [ ]:
# IMU files > load the IMU files from folder
folder_path_imu ='C:/Users/val-c/Desktop/movesense_lblled_rsp_imu'
# Get the list of all files in the folder_location
file_list = os.listdir(folder_path_imu)
# Loop via the list of files & read them into pd
dataframes = []
for file_name in file_list:
    if file_name.endswith(".csv"):  # files in csv format
        file_path = os.path.join(folder_path_imu, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
# Combine all the data_frame into one
df_imu = pd.concat(dataframes, ignore_index=True)
df_imu.head(n=5)

In [ ]:
# display nos of row & columns
#df_imu.shape

In [ ]:
# Save the merged DataFrame (df_imu) to a CSV file
#df_imu.to_csv('imu_file.csv', index=False)

- Load the combined file (ecg +imu)

In [ ]:
# Import the combined file (imu + ecg) of the movesense, performed on microsoft excel
file='C:/Users/val-c/Desktop/combined_file.csv'
combined_data=pd.read_csv(file)
combined_data

In [ ]:
# conversion of the ecg sample to mV
## defining vaiables below:
peak_to_peak_volt = 60  # dynamic range/full-scale input voltage in mV
digital_resolution = 15  # resolution
full_scale_digital_value = 2 ** digital_resolution  # number of steps in the digital output
full_scale_input_volt = peak_to_peak_volt / 2  # average voltage level in mV
digital_output_value = combined_data['sample']  # digital output values of ECG measurements
# Convert digital output values to mV
combined_data.insert(3, 'ecg_mV', (digital_output_value / full_scale_digital_value) * full_scale_input_volt) #put the new field to index position
# Display the df
combined_data.head()

In [ ]:
# drop 'gx','gy','gz', 'sample' columns
combined_data=combined_data.drop(['gx','gy','gz','sample'],axis=1)
combined_data # display dataframe

In [ ]:
# check for missing values (NaN)
missing_value_count =combined_data.isnull().sum()
missing_value_count # display the nos of missing values in the columns

In [ ]:
# Creating a new DataFrame with desired column order for acceleration measurements and introduce negative signs
combined_data_new = combined_data[['y', 'z', 'x','ecg_mV','classes']].copy()
combined_data_new[['y', 'z', 'x']] = -combined_data_new[['y', 'z', 'x']] # desired column order to match the training features > x,y,z columns
# Display the modified DataFrame for the first-five rows
combined_data_new.head()

In [ ]:
# display info of the combined file
combined_data_new.info()

In [ ]:
# Statistics of the test data dataframe
combined_data_new.describe().T

- Data visualization of the movesense measurements ( acc. & ecg)

In [ ]:
activity_map_imu = {1:'Standing still(1 min)',2:'Sitting and relaxing (1 min)',3:'Lying down (1 min)',4:'Walking (1 min)',
    5: 'Climbing stairs (1 min)',6:'Waist bends forward (20x)',7:'Frontal elevation of arms (20x)',
    8:'Knees bending (crouching) (20x)',9:'Cycling', 10:'Jogging (1 min)',11:'Running (1 min)',12:'Jump front & back (20x)'}

fig, axes = plt.subplots(6,2, figsize=(15, 15), dpi=100) # defining axes and figure size

# plot the acceleration for the different activities performed with the movesense device
for i, ax in zip(activity_map_imu.keys(), axes.flat):
    ax.plot(combined_data_new[combined_data_new['classes']==i].reset_index(drop=True)['y'], c='r', alpha=.9, label='acc. y')
    ax.plot(combined_data_new[combined_data_new['classes']==i].reset_index(drop=True)['z'], c='b', alpha=.9, label='acc. z')
    ax.plot(combined_data_new[combined_data_new['classes']==i].reset_index(drop=True)['x'], c='y', alpha=.9, label='acc. x')
    ax.set_xlabel('\nSample points', fontsize=10) #x_label
    ax.set_ylabel('\nAcceleration (m/s^2)', fontsize=10)  #y_label
    ax.set_title(f'\n{activity_map_imu[i]} - Movesense dataset', fontsize=11) # title
    ax.legend(loc="upper left", fontsize=9)

# display plots
plt.tight_layout()
plt.show()

In [ ]:
activity_map_ecg = {1:'Standing still (1 min)',2:'Sitting and relaxing (1 min)',3:'Lying down (1 min)',4:'Walking (1 min)',
    5: 'Climbing stairs (1 min)',6:'Waist bends forward (20x)',7:'Frontal elevation of arms (20x)',
    8:'Knees bending (crouching) (20x)',9:'Cycling',  10:'Jogging (1 min)',11:'Running (1 min)',12:'Jump front & back (20x)'}
# plot the ecg for the different activities performed with the movesense device
fig, axs = plt.subplots(6, 2, figsize=(15, 15), dpi=100) # defining axes and figure size
for i, ax in zip(activity_map_ecg.keys(), axs.flat):
# Plot data
    ax.plot(combined_data_new[combined_data_new['classes']==i].reset_index(drop=True)['ecg_mV'], c='r', alpha=.9, label='ECG sample')
    ax.set_xlabel('\nSample points', fontsize=10)# x_label
    ax.set_ylabel('\nAmplitude (mV)', fontsize=10) # y_label
    ax.set_title(f'\n{activity_map_ecg[i]} - Movesense dataset', fontsize=11)
    ax.legend(fontsize=9,loc="upper left")
#display plot
plt.tight_layout()
plt.show()

- Filtering the testing data

In [ ]:
# Columns of the dataframe to apply filtering
col=['y','z','x','ecg_mV']
# filtering the sensor signal from noise
# low pass filtering > remove high frequency noise
## Defining the low-pass filter parameters
cutoff_freq = 20 # hz
fs = 50  # hz
nyquist_freq = 0.5 * fs
n_order = 3 #filter_order
# Designing the Butterworth filter
b, a = butter(n_order , cutoff_freq/nyquist_freq, btype='low')
# Applying the filter to the data_columns
filtered_data = filtfilt(b, a,combined_data_new[col].values, axis=0)
combined_data_new[col] = filtered_data #replace the filtered data to the original column
# display df of the filtered data
combined_data_new[col].head()

In [ ]:
# Assign back the 'classes' column
combined_data_new['classes'] = combined_data_new['classes']
combined_data_new.head()

- Feature selection

In [ ]:
# Feature selected (y,z,x,ecg_mV) and label of the test data
_feature_ = combined_data_new.copy() #features /input
_label_ = _feature_.pop('classes') # labels/output
# display feature and label shape
_feature_.shape , _label_.shape

In [ ]:
# Display columns of the features (test data)
_feature_.columns

- Normalize the data

In [ ]:
# Creating an instance of StandardScaler to normalize the test_data (input)
scaler = StandardScaler()
# Fit and transform the feature variables
feature_scaled = scaler.fit_transform(_feature_)

- Apply data segmentation function to test data

In [ ]:
# Creating the testing_data sequence using the already defined function initially
X_test_seq,y_test_seq = create_sequences(feature_scaled, _label_, num_time_step=100,stepsize=50)
print (X_test_seq.shape, y_test_seq.shape)

- Evaluate the model on the test data

In [ ]:
# Model evaluation on the movesense_test_data to ascertain the trained model's performance
loss, accuracy =model.evaluate(X_test_seq,y_test_seq,verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'\nTest accuracy: {accuracy:.4f}')

In [ ]:
# Prediction testing on the test_data to get classification result
classify_test_data =model.predict(X_test_seq,verbose=0)
classify_test_data= np.argmax(classify_test_data, axis=1)

In [ ]:
# Create confusion matrix for the classifications > test data (acceleration + ECG)
cm = confusion_matrix(y_test_seq, classify_test_data)
class_labels  = ['Null','Standing still','Sitting and relaxing','Lying down','Walking', 'Climbing stairs',
                       'Waist bends forward', 'Frontal elevation of arms','Knees bending (crouching)','Cycling',
                      'Jogging','Running','Jump front & back' ] #'Null'
# Plot the confusion matrix for the classification >> test data
plt.figure(figsize=(12, 9),dpi=75)
sns.heatmap(cm,fmt="d",annot=True, cmap='Blues', xticklabels=class_labels, yticklabels=class_labels,linewidths = 0.1)
plt.title('\nConfusion Matrix : Classification-testing', fontsize=14, fontweight='bold')  #title
plt.xlabel('\nPredicted Activities', fontsize=12, fontweight='bold') #x-axis label
plt.ylabel('\nActual Activities', fontsize=12, fontweight='bold') #y-axis label
#display cm plot
plt.show()

- Comparison plot between the training data(Mhealth) and the test data (Movesense)

In [ ]:
# Performed activities >>
# To be used for for 1min/20x
activity_map = {1:'Standing still (1 min)',2:'Sitting and relaxing (1 min)',3:'Lying down (1 min)',
                    4:'Walking (1 min)', 5: 'Climbing stairs (1 min)',6:'Waist bends forward (20x)',
                    7:'Frontal elevation of arms (20x)',8:'Knees bending (crouching) (20x)',9:'Cycling (1 min)',
                    10:'Jogging (1 min)',11:'Running (1 min)',12:'Jump front & back (20x)'}

# To be used for 100 to 500 samples plot
_activity_map_ = {1:'Standing still',2:'Sitting and relaxing',3:'Lying down',4:'Walking',
                      5:'Climbing stairs',6:'Waist bends forward',7:'Frontal elevation of arms',
                      8:'Knees bending (crouching)',9:'Cycling',10:'Jogging',11:'Running',
                      12:'Jump front & back'}

In [ ]:
# Plot the activities performed for both mhealth and movesense datasets
fig, axs = plt.subplots(len(activity_map), 2, figsize=(12, 4 * len(activity_map)))
# Plot the acceleration measurement performed for the movesense dataset
for i, activity_id in enumerate(activity_map.keys()):
    axs[i, 1].plot(combined_data_new[combined_data_new['classes'] == activity_id].reset_index(drop=True)['y'], c='r', alpha=0.7, label='acc. y')
    axs[i, 1].plot(combined_data_new[combined_data_new['classes'] == activity_id].reset_index(drop=True)['z'], c='b', alpha=0.7, label='acc. z')
    axs[i, 1].plot(combined_data_new[combined_data_new['classes'] == activity_id].reset_index(drop=True)['x'], c='y', alpha=0.7, label='acc. x')
    axs[i, 1].set_xlabel('Sample points', fontsize=9.5)  # X label
    axs[i, 1].set_ylabel('Acceleration (m/s^2)', fontsize=10)  # Y label
    axs[i, 1].set_title(f'{activity_map[activity_id]} - Movesense Dataset', fontsize=11)  # Title
    axs[i, 1].legend(loc='upper left', fontsize=9)

# plot the chest_accleration for a subject chosen from the list of subjects for the activities of 1min/20x
## select a subject from the list of subjects in the experiment
filtered_movement_data['subject'] = movement_data['subject'] # add the subject column to the filtered data
subject = filtered_movement_data[filtered_movement_data['subject'] == 'subject_f'] #subject_f=subject-6
readings = ['acc']
for i, activity_id in enumerate(activity_map.keys()):
    for r in readings:
        if r == 'acc':
            # Plot the activities
            axs[i, 0].plot(subject[subject['classes'] == activity_id].reset_index(drop=True)[r + '_CHsensor_X'], color='r', alpha=0.7)
            axs[i, 0].plot(subject[subject['classes'] == activity_id].reset_index(drop=True)[r + '_CHsensor_Y'], color='b', alpha=0.7)
            axs[i, 0].plot(subject[subject['classes'] == activity_id].reset_index(drop=True)[r + '_CHsensor_Z'], color='y', alpha=0.7)
            axs[i, 0].set_title(f'{activity_map[activity_id]} - MHEALTH Dataset', fontsize=11)
            axs[i, 0].set_ylabel('\nAcceleration (m/s^2)', fontsize=10)
            axs[i, 0].set_xlabel('Sample points', fontsize=9.5)
            axs[i, 0].legend(["acc. x", "acc. y", "acc. z"], fontsize=9, loc="upper left")
#display plots
plt.tight_layout()
plt.show()

In [ ]:
# Plot and display peformed activities "after filtering has been done" on the datasets (mhealth and movesense measurements)
# create a plot for the acceleration measurements for the Movesense dataset 100 to 500 samples to the re-ordered columns
fig, axs = plt.subplots(len(_activity_map_), 2, figsize=(12,4 * len(_activity_map_)))

for idx, i in enumerate(_activity_map_.keys()):
    data_subset = combined_data_new[combined_data_new['classes'] == i].reset_index(drop=True)[100:501]  # select samples 100 to 500
    axs[idx, 1].plot(data_subset['y'], c='r', alpha=0.9, label='acc. y')  # acc.y
    axs[idx, 1].plot(data_subset['z'], c='b', alpha=0.9, label='acc. z')  # acc.z
    axs[idx, 1].plot(data_subset['x'], c='y', alpha=0.9, label='acc. x')  # acc.x
    axs[idx, 1].set_xlabel('Sample points', fontsize=9.5)  # x_label
    axs[idx, 1].set_ylabel('Acceleration (m/s^2)', fontsize=10)  # y_label
    axs[idx, 1].set_title(f'{_activity_map_[i]} -  Movesense Dataset', fontsize=11)  # title
    axs[idx, 1].legend(loc='upper left', fontsize=9)

# Select a subject from the list of subjects in the experiment
subject = filtered_movement_data[filtered_movement_data['subject'] == 'subject_f']
readings = ['acc']
# Plot the acceleration measurements for the Mhealth datasets for samples 100 to 500
for idx, (activity_id, activity_name) in enumerate(_activity_map_.items()):
    for r in readings:
        # acceleration chest sensor activity
        if r == 'acc':
            data_subset = subject[subject['classes'] == activity_id].reset_index(drop=True)[100:501]  # Select samples 100 to 500
            axs[idx, 0].plot(data_subset[r + '_CHsensor_X'], color='r', alpha=0.9)  # plot acc.x
            axs[idx, 0].plot(data_subset[r + '_CHsensor_Y'], color='b', alpha=0.9)  # plot acc.y
            axs[idx, 0].plot(data_subset[r + '_CHsensor_Z'], color='y', alpha=0.9)  # plot acc.z
            axs[idx, 0].set_title(f'{activity_name} - MHEALTH Dataset', fontsize=11)  # title
            axs[idx, 0].set_ylabel('\nAcceleration (m/s^2)', fontsize=10)  # y_label
            axs[idx, 0].set_xlabel('Sample points', fontsize=9.5)  # x_label
            axs[idx, 0].legend(["acc. x", "acc. y", "acc. z"], fontsize=9, loc="upper left")
# Display plots
plt.tight_layout()
plt.show()

In [ ]:
# Plot the activities performed for both mhealth and movesense datasets >1min/20x
fig, axs = plt.subplots(len(activity_map), 2, figsize=(12, 4 * len(activity_map)))
# Plot the ecg measurement performed activities for the movesense dataset
for i, activity_id in enumerate(activity_map.keys()):
    axs[i, 1].plot(combined_data_new[combined_data_new['classes'] == activity_id].reset_index(drop=True)['ecg_mV'], c='r', alpha=0.7, label='ECG Sample')
    axs[i, 1].set_xlabel('Sample points', fontsize=9.5)  # X label
    axs[i, 1].set_ylabel('Amplitude (mV)', fontsize=10)  # Y label
    axs[i, 1].set_title(f'{activity_map[activity_id]} - Movesense Dataset', fontsize=11)  # Title
    axs[i, 1].legend(loc='upper left', fontsize=9)

# plot the ecg for a subject chosen from the list of subjects for the activities of 1min/20x for'mean_ecg column'
## select a subject from the list of subjects in the experiment
subject = filtered_movement_data[filtered_movement_data['subject'] == 'subject_f'] #subject_f=subject-6
readings = ['Mean']
for i, activity_id in enumerate(activity_map.keys()):
    for r in readings:
        if r == 'Mean':
            # Plot the activities
            axs[i, 0].plot(subject[subject['classes'] == activity_id].reset_index(drop=True)[r + '_ecg'], c='purple', alpha=0.7)
            axs[i, 0].set_title(f'{activity_map[activity_id]} - MHEALTH Dataset', fontsize=11)
            axs[i, 0].set_ylabel('\nAmplitude (mV)', fontsize=10)
            axs[i, 0].set_xlabel('Sample points', fontsize=9.5)
            axs[i, 0].legend(["Mean of ECG Lead 1 & 2"], fontsize=9, loc="upper left")
# Display plots
plt.tight_layout()
plt.show()

In [ ]:
# Plot for 100 to 500 samples
fig, axs = plt.subplots(len(_activity_map_), 2, figsize=(12, 4 * len(_activity_map_)))
# Plot the ecg activites performed for the movesense dataset
for i, activity_id in enumerate(_activity_map_.keys()):
    axs[i, 1].plot(combined_data_new[combined_data_new['classes'] == activity_id].reset_index(drop=True)['ecg_mV'][100:501], c='r', alpha=0.7, label='ECG Sample')
    axs[i, 1].set_xlabel('Sample points', fontsize=9.5)  # X label
    axs[i, 1].set_ylabel('Amplitude (mV)', fontsize=10)  # Y label
    axs[i, 1].set_title(f'{_activity_map_[activity_id]} - Movesense Dataset', fontsize=11)  # Title
    axs[i, 1].legend(loc='upper left', fontsize=9)

# Select a subject from the list of subjects in the experiment for the 'mean_ecg column'
subject = filtered_movement_data[filtered_movement_data['subject'] == 'subject_f'] #subject_f=subject-6
readings = ['Mean']
for i, activity_id in enumerate(_activity_map_.keys()):
    for r in readings:
        if r == 'Mean':
            # Plot the ecg activities from Mhealth dataset
            axs[i, 0].plot(subject[subject['classes'] == activity_id].reset_index(drop=True)[r + '_ecg'][100:501], color='purple', alpha=0.7)
            axs[i, 0].set_title(f'{_activity_map_[activity_id]} - MHEALTH Dataset', fontsize=11)
            axs[i, 0].set_ylabel('\nAmplitude (mV)', fontsize=10)
            axs[i, 0].set_xlabel('Sample points', fontsize=9.5)
            axs[i, 0].legend(["Mean of ECG Lead 1 & 2"], fontsize=9, loc="upper left")

# Display plots
plt.tight_layout()
plt.show()